In [1]:
import pandas as pd
import numpy as np

# ===============================
# LOAD DATA
# ===============================
bills = pd.read_csv("bills.csv")
loans = pd.read_csv("loan_snapshots.csv")
customers = pd.read_csv("customers.csv")
tasks = pd.read_csv("tasks.csv")
task_part = pd.read_csv("task_participants.csv")

# =====================================================
# FEATURE ENGINEERING - BILLS (Repayment Behaviour)
# =====================================================
bills['bill_paid_date'] = pd.to_datetime(bills['bill_paid_date'], errors='coerce')
bills['bill_scheduled_date'] = pd.to_datetime(bills['bill_scheduled_date'], errors='coerce')

# Delay per bill
bills['bill_gap_days'] = (bills['bill_paid_date'] - bills['bill_scheduled_date']).dt.days
bills['bill_gap_days'] = bills['bill_gap_days'].fillna(0)

# Late flag
bills['is_bill_late'] = (bills['bill_gap_days'] > 0).astype(int)

# Aggregasi per loan
bill_agg = bills.groupby("loan_id").agg(
    avg_bill_gap=("bill_gap_days", "mean"),
    late_ratio=("is_bill_late", "mean"),
    bill_total_amount=("amount", "sum"),
    bill_total_paid=("paid_amount", "sum")
).reset_index()

bill_agg['paid_ratio'] = bill_agg['bill_total_paid'] / bill_agg['bill_total_amount']
bill_agg['paid_ratio'] = bill_agg['paid_ratio'].fillna(0)

# =====================================================
# FEATURE ENGINEERING - LOAN SNAPSHOT (Collection)
# =====================================================
loans["outstanding_ratio"] = loans["outstanding_amount"] / loans["principal_amount"]

# COLLECTION TARGET — Predict Who Will Deteriorate
loans["collection_target"] = (loans["dpd"] >= 7).astype(int)

# =====================================================
# FEATURE ENGINEERING - CUSTOMERS
# =====================================================
customers["date_of_birth"] = pd.to_datetime(customers["date_of_birth"], errors="coerce")
customers["age"] = (pd.Timestamp("today") - customers["date_of_birth"]).dt.days // 365
customers["age"] = customers["age"].clip(lower=18, upper=80)

# Age to Category
def age_to_group(x):
    if x <= 25: return "young"
    if x <= 35: return "adult"
    if x <= 50: return "mature"
    return "senior"

customers["age_group"] = customers["age"].apply(age_to_group)

# =====================================================
# FEATURE ENGINEERING - TASKS
# =====================================================
task_full = task_part.merge(tasks, on="task_id", how="left")

task_agg = task_full.groupby("participant_id").agg(
    task_count=("task_id", "count"),
    collection_visit_count=("task_type", lambda x: (x == "COLLECTION").sum()),
    face_match_fail_ratio=("is_face_matched", lambda x: 1 - x.mean() if x.notna().any() else 0),
    qr_match_fail_ratio=("is_qr_matched", lambda x: 1 - x.mean() if x.notna().any() else 0),
    field_payment_sum=("payment_amount", "sum")
).reset_index()

task_agg.rename(columns={"participant_id": "customer_number"}, inplace=True)
task_agg["field_payment_sum"] = task_agg["field_payment_sum"].fillna(0)

# =====================================================
# MERGE SEMUA
# =====================================================
df = loans.merge(bill_agg, on="loan_id", how="left")
df = df.merge(customers[["customer_number", "marital_status", "age_group"]], on="customer_number", how="left")
df = df.merge(task_agg, on="customer_number", how="left")

# CLEAN NA
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

print("Final for COLLECTION scoring:", df.shape)
df.head()


Final for COLLECTION scoring: (12055, 19)


,customer_number,loan_id,principal_amount,outstanding_amount,dpd,outstanding_ratio,collection_target,avg_bill_gap,late_ratio,bill_total_amount,bill_total_paid,paid_ratio,marital_status,age_group,task_count,collection_visit_count,face_match_fail_ratio,qr_match_fail_ratio,field_payment_sum
0,bfcd935f8d85aa0bf65ba718ca61c475ed61eac17916cc...,84d0afe14b1746a2b5fe0c71e3b4f9a24b09666ca3c0d3...,7000000.0,2940000.0,0,0.420000,0,-11.300000,0.140000,9255000.0,9255000.0,1.000000,MARRIED,young,0.0,0.0,0.0,0.0,0.0
1,c90ec7423eedd81d550add8e50e0174c19ebd9d74afda9...,9bb95da74486e46f1e1fbd2a5adb6502c05fac14dd66ab...,7000000.0,3144734.0,15,0.449248,1,7.979167,0.354167,9256000.0,7246466.0,0.782894,MARRIED,senior,0.0,0.0,0.0,0.0,0.0
2,af753bf0e663c9c72381e1a86e761e77620316a9defc62...,0e0fba61bd538f8baabb3132a2d0d7dfc95146ceabe429...,8000000.0,3200000.0,0,0.400000,0,-24.380000,0.100000,10655000.0,10655000.0,1.000000,MARRIED,mature,0.0,0.0,0.0,0.0,0.0
3,eb9076443e10c3880431e39e7418393c0cf67a011de990...,3632ae4fc5c7219cd573c810723ba1fdbd1ab2687f2527...,5000000.0,2300000.0,0,0.460000,0,-0.460000,0.100000,6685000.0,4545800.0,0.680000,MARRIED,mature,0.0,0.0,0.0,0.0,0.0
4,9c6311fc80d245378b38e4a592c1509c0abd90fd4ec0c1...,da2663aa307c6b24f0e2caab324b173533cff6822a1484...,12000000.0,5520000.0,12,0.460000,1,6.720000,0.620000,15365000.0,13521200.0,0.880000,MARRIED,senior,0.0,0.0,0.0,0.0,0.0


In [2]:
df.to_csv("raw_master.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_number         12055 non-null  object 
 1   loan_id                 12055 non-null  object 
 2   principal_amount        12055 non-null  float64
 3   outstanding_amount      12055 non-null  float64
 4   dpd                     12055 non-null  int64  
 5   outstanding_ratio       12055 non-null  float64
 6   collection_target       12055 non-null  int32  
 7   avg_bill_gap            12055 non-null  float64
 8   late_ratio              12055 non-null  float64
 9   bill_total_amount       12055 non-null  float64
 10  bill_total_paid         12055 non-null  float64
 11  paid_ratio              12055 non-null  float64
 12  marital_status          12055 non-null  object 
 13  age_group               12055 non-null  object 
 14  task_count              12055 non-null

In [5]:
# ============================
# 1) NUMERIC AGGREGATION
# ============================

customer_agg = df.groupby("customer_number").agg({
    "principal_amount": "sum",
    "outstanding_ratio": "mean",
    "dpd": "max",
    "collection_target": "max",         # <-- TARGET untuk COLLECTION SCORING
    "avg_bill_gap": "mean",
    "late_ratio": "mean",
    "paid_ratio": "mean",
    "task_count": "sum",
    "collection_visit_count": "sum",
    "face_match_fail_ratio": "mean",
    "qr_match_fail_ratio": "mean",
    "field_payment_sum": "sum"
})


# ============================
# 2) CATEGORICAL AGGREGATION
# ============================

cat_df = df.groupby("customer_number").agg({
    "marital_status": lambda x: x.mode().iat[0] if not x.mode().empty else None,
    "age_group":      lambda x: x.mode().iat[0] if not x.mode().empty else None
})


# ============================
# 3) MERGE KEDUANYA
# ============================

customer_final = customer_agg.join(cat_df, how="left").reset_index()

print(customer_final.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12005 entries, 0 to 12004
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_number         12005 non-null  object 
 1   principal_amount        12005 non-null  float64
 2   outstanding_ratio       12005 non-null  float64
 3   dpd                     12005 non-null  int64  
 4   collection_target       12005 non-null  int64  
 5   avg_bill_gap            12005 non-null  float64
 6   late_ratio              12005 non-null  float64
 7   paid_ratio              12005 non-null  float64
 8   task_count              12005 non-null  float64
 9   collection_visit_count  12005 non-null  float64
 10  face_match_fail_ratio   12005 non-null  float64
 11  qr_match_fail_ratio     12005 non-null  float64
 12  field_payment_sum       12005 non-null  float64
 13  marital_status          12005 non-null  object 
 14  age_group               12005 non-null

In [6]:
customer_final.head()

,customer_number,principal_amount,outstanding_ratio,dpd,collection_target,avg_bill_gap,late_ratio,paid_ratio,task_count,collection_visit_count,face_match_fail_ratio,qr_match_fail_ratio,field_payment_sum,marital_status,age_group
0,00021de397e26002b12c5e4f26451ca22b579dfec52b6b...,4500000.0,0.42000,0,0,-6.22,0.60,1.000000,0.0,0.0,0.0,0.0,0.0,MARRIED,adult
1,0009cd202de1bc3adaaf0c4d9a2a5f9a9e55bc244091fb...,5000000.0,0.46000,0,0,-0.68,0.08,0.880000,0.0,0.0,0.0,0.0,0.0,MARRIED,senior
2,000b1ec95a9fa760b750d7556ff572f482d82607642328...,5000000.0,0.62674,47,1,-0.80,0.06,0.374959,0.0,0.0,0.0,0.0,0.0,MARRIED,adult
3,000df65014c496b53101cce0083ae68e4236a5d09d0c65...,8000000.0,0.44000,0,0,-6.62,0.12,1.000000,0.0,0.0,0.0,0.0,0.0,MARRIED,young
4,001255a9e1ee84209df537b57ceecda80547690e922666...,8000000.0,0.44000,0,0,-0.56,0.42,0.820000,0.0,0.0,0.0,0.0,0.0,MARRIED,mature


In [7]:
cardinality = (
    customer_final.nunique()
      .sort_values(ascending=False)
      .reset_index()
)

cardinality.columns = ["column", "unique_values"]

print(cardinality)


                    column  unique_values
0          customer_number          12005
1             avg_bill_gap           1855
2               paid_ratio           1336
3        outstanding_ratio            855
4                      dpd            146
5               late_ratio             99
6         principal_amount             28
7                age_group              4
8        collection_target              2
9           marital_status              2
10              task_count              1
11  collection_visit_count              1
12   face_match_fail_ratio              1
13     qr_match_fail_ratio              1
14       field_payment_sum              1


In [8]:
customer_final_banget = customer_final.drop(
    ['collection_visit_count','face_match_fail_ratio', 'qr_match_fail_ratio', 'field_payment_sum'],
    axis=1
)

In [9]:
customer_final_banget.head()

,customer_number,principal_amount,outstanding_ratio,dpd,collection_target,avg_bill_gap,late_ratio,paid_ratio,task_count,marital_status,age_group
0,00021de397e26002b12c5e4f26451ca22b579dfec52b6b...,4500000.0,0.42000,0,0,-6.22,0.60,1.000000,0.0,MARRIED,adult
1,0009cd202de1bc3adaaf0c4d9a2a5f9a9e55bc244091fb...,5000000.0,0.46000,0,0,-0.68,0.08,0.880000,0.0,MARRIED,senior
2,000b1ec95a9fa760b750d7556ff572f482d82607642328...,5000000.0,0.62674,47,1,-0.80,0.06,0.374959,0.0,MARRIED,adult
3,000df65014c496b53101cce0083ae68e4236a5d09d0c65...,8000000.0,0.44000,0,0,-6.62,0.12,1.000000,0.0,MARRIED,young
4,001255a9e1ee84209df537b57ceecda80547690e922666...,8000000.0,0.44000,0,0,-0.56,0.42,0.820000,0.0,MARRIED,mature


In [10]:
customer_final_banget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12005 entries, 0 to 12004
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_number    12005 non-null  object 
 1   principal_amount   12005 non-null  float64
 2   outstanding_ratio  12005 non-null  float64
 3   dpd                12005 non-null  int64  
 4   collection_target  12005 non-null  int64  
 5   avg_bill_gap       12005 non-null  float64
 6   late_ratio         12005 non-null  float64
 7   paid_ratio         12005 non-null  float64
 8   task_count         12005 non-null  float64
 9   marital_status     12005 non-null  object 
 10  age_group          12005 non-null  object 
dtypes: float64(6), int64(2), object(3)
memory usage: 1.0+ MB


In [11]:
cardinality = (
    customer_final_banget.nunique()
      .sort_values(ascending=False)
      .reset_index()
)

cardinality.columns = ["column", "unique_values"]

print(cardinality)


               column  unique_values
0     customer_number          12005
1        avg_bill_gap           1855
2          paid_ratio           1336
3   outstanding_ratio            855
4                 dpd            146
5          late_ratio             99
6    principal_amount             28
7           age_group              4
8   collection_target              2
9      marital_status              2
10         task_count              1


In [12]:
# customer_final_banget = customer_final_banget.drop(['collection_visit_count','face_match_fail_ratio', 'qr_match_fail_ratio', 'field_payment_sum'], axis=1)
customer_final_banget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12005 entries, 0 to 12004
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_number    12005 non-null  object 
 1   principal_amount   12005 non-null  float64
 2   outstanding_ratio  12005 non-null  float64
 3   dpd                12005 non-null  int64  
 4   collection_target  12005 non-null  int64  
 5   avg_bill_gap       12005 non-null  float64
 6   late_ratio         12005 non-null  float64
 7   paid_ratio         12005 non-null  float64
 8   task_count         12005 non-null  float64
 9   marital_status     12005 non-null  object 
 10  age_group          12005 non-null  object 
dtypes: float64(6), int64(2), object(3)
memory usage: 1.0+ MB


In [13]:
customer_final_banget.head()

,customer_number,principal_amount,outstanding_ratio,dpd,collection_target,avg_bill_gap,late_ratio,paid_ratio,task_count,marital_status,age_group
0,00021de397e26002b12c5e4f26451ca22b579dfec52b6b...,4500000.0,0.42000,0,0,-6.22,0.60,1.000000,0.0,MARRIED,adult
1,0009cd202de1bc3adaaf0c4d9a2a5f9a9e55bc244091fb...,5000000.0,0.46000,0,0,-0.68,0.08,0.880000,0.0,MARRIED,senior
2,000b1ec95a9fa760b750d7556ff572f482d82607642328...,5000000.0,0.62674,47,1,-0.80,0.06,0.374959,0.0,MARRIED,adult
3,000df65014c496b53101cce0083ae68e4236a5d09d0c65...,8000000.0,0.44000,0,0,-6.62,0.12,1.000000,0.0,MARRIED,young
4,001255a9e1ee84209df537b57ceecda80547690e922666...,8000000.0,0.44000,0,0,-0.56,0.42,0.820000,0.0,MARRIED,mature


In [14]:
customer_final_banget.to_csv("customer_risk2.csv")